In [1]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
from nltk.corpus import stopwords
import string
import re


In [2]:
data=list(nltk.corpus.treebank.tagged_sents())


In [3]:
train_set,test_set=train_test_split(data,train_size=0.01,test_size=0.99)

In [4]:
train_words=[tup[0] for sent in train_set for tup in sent]
test_words=[tup[0] for sent in test_set for tup in sent]
Ntrain = [word for word in train_words if word.isalpha()]
lowerTrain = [word.lower() for word in Ntrain]
Ntest = [word for word in test_words if word.isalpha()]
lowerTest = [word.lower() for word in Ntest]
print(lowerTrain)
print(len(train_words))
print(len(lowerTrain))
print(len(test_words))
print(len(lowerTest))


['the', 'court', 'hearing', 'began', 'in', 'early', 'october', 'at', 'the', 'request', 'of', 'anthony', 'hazell', 'district', 'auditor', 'for', 'hammersmith', 'who', 'argued', 'that', 'local', 'councils', 'are', 'vested', 'with', 'constitutional', 'authority', 'to', 'engage', 'in', 'such', 'activities', 'but', 'the', 'strength', 'in', 'heating', 'oil', 'helped', 'push', 'up', 'crude', 'oil', 'they', 'take', 'place', 'in', 'government', 'programs', 'that', 'seem', 'for', 'corruption', 'odyssey', 'partners', 'limited', 'partnership', 'an', 'investment', 'firm', 'completed', 'the', 'purchase', 'of', 'may', 'department', 'stores', 'caldor', 'discount', 'chain', 'for', 'million', 'plus', 'the', 'assumption', 'of', 'million', 'in', 'debt', 'the', 'white', 'house', 'said', 'bush', 'decided', 'to', 'grant', 'status', 'for', 'categories', 'but', 'turned', 'down', 'such', 'treatment', 'for', 'other', 'types', 'of', 'watches', 'because', 'of', 'the', 'potential', 'for', 'material', 'injury', 'to'

In [5]:
print(train_words)


['The', 'court', 'hearing', 'began', 'in', 'early', 'October', 'at', 'the', 'request', 'of', 'Anthony', 'Hazell', ',', 'district', 'auditor', 'for', 'Hammersmith', ',', 'who', '*T*-63', 'argued', 'that', 'local', 'councils', 'are', "n't", 'vested', 'with', 'constitutional', 'authority', '*', 'to', 'engage', 'in', 'such', 'capital-markets', 'activities', '.', 'But', 'the', 'strength', 'in', 'heating', 'oil', 'helped', '*-1', 'push', 'up', 'crude', 'oil', '.', 'They', 'take', 'place', 'in', 'government', 'programs', 'that', '*T*-1', 'seem', 'tailor-made', 'for', 'corruption', '.', 'Odyssey', 'Partners', 'Limited', 'Partnership', ',', 'an', 'investment', 'firm', ',', 'completed', 'the', 'purchase', 'of', 'May', 'Department', 'Stores', 'Co.', "'s", 'Caldor', 'discount', 'chain', 'for', '$', '500', 'million', '*U*', 'plus', 'the', 'assumption', 'of', '$', '52', 'million', '*U*', 'in', 'debt', '.', 'The', 'White', 'House', 'said', '0', 'Mr.', 'Bush', 'decided', '*-1', 'to', 'grant', 'duty-fr

In [6]:
train_vocab_set=set(lowerTrain)
print(len(train_vocab_set))
len_train_vocab_set=len(train_vocab_set)
#print(train_vocab_set)

445


In [7]:
def t2_given_t1(t2,t1,train_bag=lowerTrain): # tags=toks  tag=tok
    
    toks=[pair for pair in train_bag]
    #unigram
    t1_toks_list=[tok for tok in toks if tok ==t1]
    t1_tok_count=len(t1_toks_list)
    #bigram
    t2_given_t1_list=[toks[index+1] for index in range(len(toks)-1) if toks[index] == t1 and toks[index+1] == t2]
    t2_given_t1_count=len(t2_given_t1_list)
    
    return(t2_given_t1_count, t1_tok_count)

In [8]:
len_train_words=len(lowerTrain)

tok_matrix=np.zeros((len_train_words,len_train_words),dtype='float32')
for i, t1 in enumerate(list(lowerTrain)):
    for j, t2 in enumerate(list(lowerTrain)):
         if (t2_given_t1(t2,t1)[1])!=0:
                  tok_matrix[i,j]=((t2_given_t1(t2,t1)[0])+1)/((t2_given_t1(t2,t1)[1])+len_train_vocab_set)
       
    

In [9]:
tok_df=pd.DataFrame(tok_matrix,columns=list(lowerTrain),index=list(lowerTrain))
tok_df

,the,court,hearing,began,in,early,october,at,the,request,...,shares,speculators,or,risk,arbitragers,dumped,every,stock,they,had
the,0.002041,0.006122,0.002041,0.002041,0.002041,0.002041,0.002041,0.002041,0.002041,0.004082,...,0.002041,0.002041,0.002041,0.002041,0.002041,0.002041,0.002041,0.002041,0.002041,0.002041
court,0.002232,0.002232,0.004464,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232,...,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232
hearing,0.002242,0.002242,0.002242,0.004484,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,...,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242
began,0.002242,0.002242,0.002242,0.002242,0.004484,0.002242,0.002242,0.002242,0.002242,0.002242,...,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242
in,0.010638,0.002128,0.002128,0.002128,0.002128,0.004255,0.002128,0.002128,0.010638,0.002128,...,0.002128,0.002128,0.002128,0.002128,0.002128,0.002128,0.002128,0.002128,0.002128,0.002128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dumped,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,...,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.004474,0.002237,0.002237,0.002237
every,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,...,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.002242,0.004484,0.002242,0.002242
stock,0.002222,0.002222,0.002222,0.002222,0.004444,0.002222,0.002222,0.002222,0.002222,0.002222,...,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.004444,0.002222
they,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,...,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.002237,0.004474


In [10]:
for i in range(len(lowerTrain)):
    ls=[]
    maxx=tok_matrix[i,0]
    mj=0;
    for j in range(len(lowerTrain)):
        if tok_matrix[i,j]>=maxx:
            maxx=tok_matrix[i,j]
            mj=j
    for k in range(len(lowerTrain)):
        if tok_matrix[i,k]==maxx:
            if lowerTrain[k] in ls:
                continue
            else:
                ls.append(lowerTrain[k])              
        
    print(lowerTrain[i]+" "+str(ls)[:])
        


the ['court', 'current']
court ['hearing', 'decision', 'has']
hearing ['began']
began ['in']
in ['the']
early ['october']
october ['at']
at ['the', 'all']
the ['court', 'current']
request ['of']
of ['the']
anthony ['hazell']
hazell ['district']
district ['auditor']
auditor ['for']
for ['hammersmith', 'such', 'corruption', 'million', 'categories', 'other', 'material', 'discretionary', 'a', 'them', 'similar', 'school', 'stock', 'changes', 'imports']
hammersmith ['who', 'and']
who ['argued', 'resigned']
argued ['that']
that ['the']
local ['councils']
councils ['are', 'workers']
are ['vested', 'even']
vested ['with']
with ['constitutional', 'even']
constitutional ['authority']
authority ['to', 'engaging']
to ['match', 'get']
engage ['in']
in ['the']
such ['as']
activities ['but']
but ['the', 'turned', 'voters', 'it']
the ['court', 'current']
strength ['in']
in ['the']
heating ['oil']
oil ['helped', 'they']
helped ['push']
push ['up']
up ['to', 'crude', 'decades']
crude ['oil']
oil ['helped

In [11]:
while True:
    inp=input()
    tok = inp.split()
    last_inp=tok[len(tok)-1]
    point=-1

    for i in range(len(lowerTrain)):
        if last_inp==lowerTrain[i]:
            point=i

    ls=[]
    maxx=tok_matrix[point,0]
    mj=0;
    for j in range(len(lowerTrain)):
        if tok_matrix[point,j]>=maxx:
            maxx=tok_matrix[point,j]
            mj=j
    for k in range(len(lowerTrain)):
        if tok_matrix[point,k]==maxx:
            if lowerTrain[k] in ls:
                continue
            else:
                ls.append(lowerTrain[k])              

    if point==-1:
        print("Not Found")
    else:
        print(lowerTrain[point]+" "+str(ls)[:])

    


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.